In [1]:
import pandas as pd
import numpy as np
import re
from math import isnan

In [2]:
raw = pd.read_json('raw.json', orient='index')

In [3]:
raw.columns

Index(['Model', 'Year', 'Category', 'Rating', 'Displacement', 'Engine type',
       'Torque', 'Bore x stroke', 'Fuel system', 'Fuel control',
       'Cooling system', 'Gearbox', 'Transmission type,final drive',
       'Frame type', 'Rake (fork angle)', 'Trail', 'Front suspension',
       'Rear suspension', 'Rear wheel travel', 'Front tyre', 'Rear tyre',
       'Front brakes', 'Diameter', 'Rear brakes', 'Seat height',
       'Ground clearance', 'Wheelbase', 'Fuel capacity', 'Color options',
       'Starter', 'Comments', 'Insurance costs', 'Finance options',
       'Parts finder', 'Ask questions', 'Related bikes', 'Price as new',
       'Engine details', 'Power', 'Compression', 'Valves per cylinder',
       'Ignition', 'Lubrication system', 'Clutch', 'Driveline',
       'Fuel consumption', 'Greenhouse gases', 'Emission details',
       'Exhaust system', 'Front wheel travel', 'Wheels', 'Dry weight',
       'Weight incl. oil, gas, etc', 'Power/weight ratio', 'Overall height',
       'Overa

In [4]:
raw.drop(columns = ['Insurance costs', 'Finance options','Parts finder', 'Ask questions', 'Related bikes', 'Dirt-bike parts', 'Motocross parts', 'ATV parts'], inplace=True)

In [5]:
def extract_two_floats(fullstring):
    try:
        numbers = re.findall(r'\d+\.\d+|\d+', fullstring)
        return float(numbers[0]),  float(numbers[1])
    except TypeError:
        return np.nan, np.nan

In [6]:
raw['Displacement ccm'], raw['Displacement cubic inches'] = zip(*raw['Displacement'].map(extract_two_floats))
raw.drop(columns=['Displacement'], inplace=True)

raw['Trail mm'], raw['Trail inches'] = zip(*raw['Trail'].map(extract_two_floats))
raw.drop(columns=['Trail'], inplace=True)

raw['Diameter mm'], raw['Diameter inches'] = zip(*raw['Diameter'].map(extract_two_floats))
raw.drop(columns=['Diameter'], inplace=True)

raw['Ground clearance mm'], raw['Ground clearance inches'] = zip(*raw['Ground clearance'].map(extract_two_floats))
raw.drop(columns=['Ground clearance'], inplace=True)

raw['Wheelbase mm'], raw['Wheelbase inches'] = zip(*raw['Wheelbase'].map(extract_two_floats))
raw.drop(columns=['Wheelbase'], inplace=True)

raw['Fuel capacity liters'], raw['Fuel capacity gallons'] = zip(*raw['Fuel capacity'].map(extract_two_floats))
raw.drop(columns=['Fuel capacity'], inplace=True)

raw['Front wheel travel mm'], raw['Front wheel travel inches'] = zip(*raw['Front wheel travel'].map(extract_two_floats))
raw.drop(columns=['Front wheel travel'], inplace=True)

raw['Dry weight kg'], raw['Dry weight pounds'] = zip(*raw['Dry weight'].map(extract_two_floats))
raw.drop(columns=['Dry weight'], inplace=True)

raw['Weight incl. oil, gas, etc kg'], raw['Weight incl. oil, gas, etc pounds'] = zip(*raw['Weight incl. oil, gas, etc'].map(extract_two_floats))
raw.drop(columns=['Weight incl. oil, gas, etc'], inplace=True)

raw['Overall height mm'], raw['Overall height inches'] = zip(*raw['Overall height'].map(extract_two_floats))
raw.drop(columns=['Overall height'], inplace=True)

raw['Overall length mm'], raw['Overall length inches'] = zip(*raw['Overall length'].map(extract_two_floats))
raw.drop(columns=['Overall length'], inplace=True)

raw['Overall width mm'], raw['Overall width inches'] = zip(*raw['Overall width'].map(extract_two_floats))
raw.drop(columns=['Overall width'], inplace=True)

raw['Reserve fuel capacity liters'], raw['Reserve fuel capacity gallons'] = zip(*raw['Reserve fuel capacity'].map(extract_two_floats))
raw.drop(columns=['Reserve fuel capacity'], inplace=True)

raw['Top speed km/h'], raw['Top speed mph'] = zip(*raw['Top speed'].map(extract_two_floats))
raw.drop(columns=['Top speed'], inplace=True)

raw['Alternate seat height mm'], raw['Alternate seat height inches'] = zip(*raw['Alternate seat height'].map(extract_two_floats))
raw.drop(columns=['Alternate seat height'], inplace=True)

raw['Oil capacity liters'], raw['Oil capacity quarts'] = zip(*raw['Oil capacity'].map(extract_two_floats))
raw.drop(columns=['Oil capacity'], inplace=True)

In [7]:
def extract_single_float(fullstring):
    try:
        return float(re.findall(r'\d+\.\d+', fullstring)[0])
    except TypeError:
        return np.nan

In [8]:
raw.rename(columns={"Compression": "Compression Ratio"}, inplace=True)
raw['Compression Enumerator'] = raw['Compression Ratio'].apply(lambda row: extract_single_float(row))

raw['1/4 mile (0.4 km)'] = raw['1/4 mile (0.4 km)'].apply(lambda row: extract_single_float(row))

raw['Power/weight ratio'] = raw['Power/weight ratio'].apply(lambda row: extract_single_float(row))

raw['Greenhouse gases'] = raw['Greenhouse gases'].apply(lambda row: extract_single_float(row))

raw['60-140 km/h (37-87 mph), highest gear'] = raw['60-140 km/h (37-87 mph), highest gear'].apply(lambda row: extract_single_float(row))

raw['0-100 km/h (0-62 mph)'] = raw['0-100 km/h (0-62 mph)'].apply(lambda row: extract_single_float(row))

In [9]:
def extract_three_floats(fullstring):
    try:
        numbers = re.findall(r'\d+\.\d+|\d+', fullstring)
        return float(numbers[0]),  float(numbers[1]), float(numbers[2])
    except TypeError:
        return np.nan, np.nan, np.nan

In [10]:
raw['Fuel consumption liters/100 km'], raw['Fuel consumption km/liter'], raw['Fuel consumption miles/gallon'] = zip(*raw['Fuel consumption'].map(extract_three_floats))
raw.drop(columns=['Fuel consumption'], inplace=True)

In [11]:
raw['Automatic gearbox'] = (raw['Gearbox'] == 'Automatic') | (raw['Gearbox'] == '2-speed automatic') | (raw['Gearbox'] == '3-speed automatic')

In [12]:
def transform_rating(fullstring):
    #Do you know this bike?Click here to rate it. We miss 2 votes to show the rating.
    #3.5 \xa0See the detailed rating of touring capabilities, reliability, accident risk, etc. Compare with any other motorbike.
    
    if re.search('Do you know this bike?', fullstring) == None:
        return float(re.search(r'\d.\d', fullstring).group())
    else:
        return np.nan

raw['Rating'] = raw['Rating'].apply(lambda row: transform_rating(row))

In [13]:
def extract_torques(fullstring):
    #58.0 Nm (5.9 kgf-m or 42.8 ft.lbs)
    #90.0 Nm (9.2 kgf-m or 66.4 ft.lbs) @ 6500 RPM
    #NaN
    
    try:
        fullstring = fullstring.replace(',', '')
        numbers = re.findall(r'\d+.\d+', fullstring)

        rpm = np.nan
        if len(numbers) == 4: rpm = int(numbers[3])
        return float(numbers[0]), float(numbers[1]), float(numbers[2]), rpm
    except AttributeError:
        if isnan(fullstring): return np.nan, np.nan, np.nan, np.nan
        
raw['Torque Nm'], raw['Torque kgf-m'], raw['Torque ft.lbs'], raw['Torque Benchmark RPM'] = zip(*raw['Torque'].map(extract_torques))
raw.drop(columns=['Torque'], inplace=True)

In [14]:
def extract_bore_stroke(fullstring):
    #100.0 x 76.4 mm (3.9 x 3.0 inches)
    #80.0 x 0.0 mm (3.1 x  inches)
    #NaN
    
    try:
        fullstring = fullstring.replace(',', '')
        numbers = re.findall(r'\d+.\d+', fullstring)
        
        if len(numbers) != 4: return float(numbers[0]), np.nan, float(numbers[2]), np.nan
        return float(numbers[0]), float(numbers[1]), float(numbers[3]), float(numbers[3])
    except AttributeError:
        if isnan(fullstring): return np.nan, np.nan, np.nan, np.nan

raw['Bore mm'], raw['Stroke mm'], raw['Bore inches'], raw['Stroke inches'] = zip(*raw['Bore x stroke'].map(extract_bore_stroke))
raw.drop(columns=['Bore x stroke'], inplace=True)

In [15]:
def extract_rake(fullstring):
    #26.5°
    #nan
    
    try:
        return float(re.match(r'\d+.\d+', fullstring).group())
    except TypeError:
        return np.nan

raw['Rake (fork angle)'] = raw['Rake (fork angle)'].apply(lambda row: extract_rake(row))   

In [16]:
def extract_colors(fullstring):
    #White, black
    #Grey/white/green
    #Racing Black, Fluo Red, Racing White\r\n
    #Dark Losail
    #NaN
    
    try:
        fullstring = fullstring.strip('\t\r\n')
        if '/' in fullstring: return fullstring.replace('/', ', ')
        return fullstring
    except AttributeError:
        return np.nan
    
raw['Color options'] = raw['Color options'].apply(lambda row: extract_colors(row))

In [17]:
def extract_power(fullstring):
    #95.2 HP (69.5 kW)) @ 8750 RPM
    #28.2 HP (20.6 kW))
    #NaN
    
    try:
        numbers = list(re.findall(r'\d+\.\d+|\d+', fullstring))
        if len(numbers) == 2: return float(numbers[0]), float(numbers[1]), np.nan
        return float(numbers[0]), float(numbers[1]), float(numbers[2])
    except TypeError:
        return np.nan, np.nan, np.nan
    
raw['Power HP'], raw['Power kW'], raw['Power Benchmark RPM'] = zip(*raw['Power'].map(extract_power))
raw.drop(columns=['Power'], inplace=True)

In [18]:
def extract_price(fullstring):
    #Euro 9990.  MSRP depend on country, taxes, accessories, etc.
    #US$ 10995.   MSRP depend on country, taxes, accessories, etc.
    #nan
    
    try:
        #if 'US$' in fullstring: return int(re.match(r'\d+', fullstring).group()), np.nan
        #if 'Euro' in fullstring: return np.nan, int(re.match(r'\d+', fullstring).group())
        
        number = int(re.findall(r'\d+', fullstring)[0])
        if 'US$' in fullstring: return number, np.nan
        if 'Euro' in fullstring: return np.nan, number
        
    except TypeError:
        return np.nan, np.nan
    
raw['Price as new USD'], raw['Price as new Euros'] = zip(*raw['Price as new'].map(extract_price))
raw.drop(columns=['Price as new'], inplace=True)

In [19]:
raw['Ignition'] = raw['Ignition'].apply(lambda row: row.strip() if pd.notnull(row) else np.nan)

In [20]:
raw.to_csv('bikes.csv')

In [21]:
summary = pd.DataFrame(columns=['Column Name', 'Unique Values', 'Null Values', 'Null Values percentage', 'Type'])

for col in raw.columns:
    #entry = pd.DataFrame([[col, raw[col].nunique(), sum(raw[col].isnull()), format(sum(raw[col].isnull())  / raw[col].count() * 100, '.2f'), raw[col].dtype]], columns=['Column Name', 'Unique Values', 'Null Values', 'Null values percentage', 'Type'])
    entry = pd.DataFrame(data = {
        'Column Name': [col],
        'Unique Values': [raw[col].nunique()],
        'Null Values': [sum(raw[col].isnull())],
        'Null Values percentage': [format(sum(raw[col].isnull())  / raw[col].count() * 100, '.2f')],
        'Type': [raw[col].dtype]
    })
    summary = summary.append(entry, ignore_index=True)

In [22]:
summary

,Column Name,Unique Values,Null Values,Null Values percentage,Type
0,Model,17143,0,0.00,object
1,Year,128,0,0.00,int64
2,Category,18,0,0.00,object
3,Rating,33,17251,80.71,float64
4,Engine type,30,10,0.03,object
...,...,...,...,...,...
93,Power HP,820,12357,47.04,float64
94,Power kW,734,12357,47.04,float64
95,Power Benchmark RPM,244,15514,67.13,float64
96,Price as new USD,1269,31993,482.48,float64


In [26]:
summary.to_csv('summary.csv')